In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

# ==========================================
# ENABLE WIDGETS IN COLAB (CRITICAL!)
# ==========================================
from google.colab import output
output.enable_custom_widget_manager()

# ==========================================
# 1. PREPARE DATA
# ==========================================
plot_df = df.copy()

# Convert columns to correct types
plot_df['Transaction Date'] = pd.to_datetime(plot_df['Transaction Date'], errors='coerce')
plot_df = plot_df.dropna(subset=['Transaction Date']).sort_values(by='Transaction Date')
plot_df['Amount'] = pd.to_numeric(plot_df['Amount'], errors='coerce').fillna(0)
plot_df['quantity'] = pd.to_numeric(plot_df['quantity'], errors='coerce').fillna(0)
plot_df['Quantity unit'] = plot_df['Quantity unit'].astype(str).str.strip().fillna('Unknown')

available_units = sorted(plot_df['Quantity unit'].unique().tolist())

# ==========================================
# 2. CREATE INTERACTIVE WIDGETS
# ==========================================
dropdown = widgets.Dropdown(
    options=available_units,
    value=available_units[0] if available_units else None,
    description='Select Unit:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

output_area = widgets.Output()

# ==========================================
# 3. PLOTTING FUNCTION
# ==========================================
def render_plot(selected_unit):
    filtered_df = plot_df[plot_df['Quantity unit'] == selected_unit]
    daily_data = filtered_df.groupby('Transaction Date')[['Amount', 'quantity']].sum().reset_index()

    output_area.clear_output(wait=True)

    with output_area:
        if daily_data.empty:
            print(f"⚠️ No data found for unit: '{selected_unit}'")
        else:
            fig = make_subplots(specs=[[{"secondary_y": True}]])

            fig.add_trace(
                go.Scatter(
                    x=daily_data['Transaction Date'],
                    y=daily_data['Amount'],
                    name="Total Amount ($)",
                    mode='lines+markers',
                    line=dict(color='#1f77b4', width=2)
                ),
                secondary_y=False,
            )

            fig.add_trace(
                go.Scatter(
                    x=daily_data['Transaction Date'],
                    y=daily_data['quantity'],
                    name=f"Quantity ({selected_unit})",
                    mode='lines+markers',
                    line=dict(color='#ff7f0e', width=2, dash='dot')
                ),
                secondary_y=True,
            )

            fig.update_layout(
                title=f"<b>Financial vs Volume Analysis</b><br>Unit: {selected_unit}",
                hovermode="x unified",
                height=500,
                legend=dict(orientation="h", y=1.1, x=0.5, xanchor="center"),
                margin=dict(l=20, r=20, t=60, b=20)
            )

            fig.update_yaxes(title_text="<b>Total Amount (USD)</b>", title_font=dict(color="#1f77b4"), secondary_y=False)
            fig.update_yaxes(title_text=f"<b>Quantity ({selected_unit})</b>", title_font=dict(color="#ff7f0e"), secondary_y=True)
            fig.update_xaxes(title_text="Transaction Date")

            # Just use fig.show() - Colab handles it automatically
            fig.show()

# ==========================================
# 4. CONNECT AND DISPLAY
# ==========================================
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        render_plot(change['new'])

dropdown.observe(on_change)

print("Select a Quantity Unit to view trends:")
display(dropdown)
display(output_area)

# Initialize
if available_units:
    render_plot(available_units[0])
else:
    print("❌ No data available in 'Quantity unit' column.")